# 2.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought

Auflösung ist ein zentraler Faktor für die OCR Qualität. Entsprechend hilft es, bei älteren Plänen den schriftlichen Teil spaltenweise als Bild abzuspeichern (Spalte = PDF-Seite). Maximale Auflösung => Fehlerquote minimieren. (siehe 2.1.1 bzgl. Nutzungsschablone)

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [ ]:
# Textteil – WITH CONTEXT
# Schema / BauNVO

In [2]:
# A) Textteil – ONE COL WITHOUT CONTEXT
tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-TT-crop.png"
ze_tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"

async def run(img_path):
    base64_img, img_size = parser.imageToBase64(img_path)
    print("image_size:", img_size)
    message = jarvis.request([
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_img}",
                "detail": "high"
            }
        },
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
    return message

messages = await asyncio.gather(run(ze_tt_path), run(tt_path))
for msg in messages:
    print("#############################################")
    print(msg)

image_size: [768, 1117]
image_size: [768, 1775]
#############################################
Hier sind die extrahierten Informationen aus dem Bebauungsplan "Grübler" zu den angeforderten Themen:

### Art der baulichen Nutzung
- **Wohngebiete (WR)**
- **Allgemeine Wohngebiete (WA)**
- **Gewerbegebiete (GE)**

### Maß der baulichen Nutzung
- **Zahl der Vollgeschosse (§ 9 (1) 1 BauNVO)**
  - Die Zahl der Vollgeschosse ist im Plan festgesetzt.
- **Grundflächenzahl (§ 9 (1) 2 BauNVO)**
  - Die Grundflächenzahl ist im Plan festgesetzt.
- **Geschossflächenzahl (§ 9 (1) 2 BauNVO)**
  - Die Geschossflächenzahl ist im Plan festgesetzt.
- **Baumassenzahl (§ 9 (1) 2 BauNVO)**
  - Die Baumassenzahl ist im Plan festgesetzt.
- **Höhe der baulichen Anlagen (§ 9 (1) 2 BauNVO)**
  - Die Höhe der baulichen Anlagen ist im Plan festgesetzt.

### Bauweise
- **Offene Bauweise (§ 22 BauNVO)**
  - Die Bauweise ist im Plan festgesetzt.
- **Geschlossene Bauweise (§ 22 BauNVO)**
  - Die Bauweise ist im Plan fest

In [3]:
# B) Textteil – MULTI COL WITHOUT CONTEXT
folder_path = "../data/processed/bpläne/1_alles_in_einem_dokument"
img_name = "/L22-01-TT-crop"
tt_prompts = parser.images2prompts(folder_path, img_name)

msg = jarvis.request([
    *tt_prompts,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
        # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
    }
])

print(msg)


### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)
1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)
1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)
Zulässig sind:
1. Wohngebäude
2. nicht störende Handwerksbetriebe
3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)
Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:
1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.

1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)
Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.

### Maß der baulichen Nutzung:
1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
Z: 1
GRZ: 0,4
GF: 0,5

### Bauweise:
1.3 Bauweise (§ 22 BauNVO)
E = offene Bauweise, nur Einzelhäuser zulässig
a = abweichende Bauweise für Garagen, die giebelständig a